https://adventofcode.com/2022/day/13

In [1]:
import re
from dataclasses import dataclass
from copy import copy
from functools import cmp_to_key
import json

def read_data_in_pairs(filename: str) -> list:
    with open(filename, 'r') as f:
        signal_pairs = [pair.split('\n') for pair in f.read().split('\n\n')]
    return signal_pairs

def read_all_signals(filename: str) -> list:
    with open(filename, 'r') as f:
        signals = [line for line in f.read().split('\n') if line]
    return signals

def parse_signal_string(signal: str) -> list:
    to_return = []
    current_list = to_return
    parent_lists = []
    signal = signal[1:-1] # strip outer list demarcation
    while len(signal) > 0:
        if signal[0] == '[':
            current_list.append([])
            parent_lists.append(current_list)
            current_list = current_list[-1]
            signal = signal[1:]
        elif signal[0] == ']':
            current_list = parent_lists.pop()
            signal = signal[1:]
        elif signal[0] == ',':
            signal = signal[1:]
        else:
            s = re.search('\d+', signal).group()
            current_list.append(int(s))
            signal = signal[len(s):]
    return to_return

def parse_signal_string_lol(signal: str) -> list:
    return json.loads(signal)
        
def compare(x, y, level=0, verbose=False):
    i = copy(x)
    j = copy(y)
    prefix = '  '*level
    if verbose:
        print(f"{prefix}- Compare {i} vs {j}")
    if isinstance(i, int) and isinstance(j, int):
        if i > j:
            if verbose:
                print(f"{prefix}- Right side is smaller, so inputs are not in the right order")
            return False
        elif i < j:
            if verbose:
                print(f"{prefix}- Left side is smaller, so inputs are in the right order")
            return True
        else:
            return None
    elif isinstance(i, list) and isinstance(j, list):
        if len(i) < len(j):
            i.extend([None]*(len(j)-len(i)))
        if len(j) < len(i):
            j.extend([None]*(len(i)-len(j)))
        for a, b in zip(i, j):
            if a == None:
                if verbose:
                    print(f"{prefix}  - Left side ran out of items, so inputs are in the right order")
                return True
            if b == None:
                if verbose:
                    print(f"{prefix}  - Right side ran out of items, so inputs are not in the right order")
                return False         
            result = compare(a, b, level=level+1, verbose=verbose)
            if result is not None:
                return result
            else:
                continue
    elif isinstance(i, int) and isinstance(j, list):
        if verbose:
            print(f'{prefix}- Mixed types; convert left to [{i}] and retry comparison')
        result = compare([i], j, level=level+1, verbose=verbose)
        return result
    elif isinstance(i, list) and isinstance(j, int):
        if verbose:
            print(f'{prefix}- Mixed types; convert right to [{j}] and retry comparison')
        result = compare(i, [j], level=level+1, verbose=verbose)
        return result

def compare_wrapper(x, y):
    if compare(x, y):
        return -1
    else:
        return 1
    
def solve_part_1(filename: str,
                 verbose: bool=False) -> int:
    signal_pairs = read_data_in_pairs(filename)
    right_order_pairs = []
    for i, pair in enumerate(signal_pairs, 1):
        if verbose:
            print(f'=== Pair {i} ===')
        s1 = parse_signal_string_lol(pair[0])
        s2 = parse_signal_string_lol(pair[1])
        right_order = compare(s1, s2, verbose=verbose)
        if right_order:
            right_order_pairs.append(i)
        if verbose:
            print()
    return sum(right_order_pairs)

def solve_part_2(filename: str) -> int:
    data = read_all_signals(filename)
    signals = [parse_signal_string_lol(s) for s in data]
    divider_packets = [[[2]], [[6]]]
    signals.extend(divider_packets)
    sorted_signals = sorted(signals, key=cmp_to_key(compare_wrapper))
    loc = [sorted_signals.index(d)+1 for d in divider_packets]
    return loc[0]*loc[1]

In [2]:
filename = "../example_data/day13_example_data.txt"
solve_part_1(filename=filename)

13

In [3]:
filename = "../data/day13_data.txt"
solve_part_1(filename=filename)

5252

In [4]:
filename = "../example_data/day13_example_data.txt"
solve_part_2(filename=filename)

140

In [5]:
filename = "../data/day13_data.txt"
solve_part_2(filename=filename)

20592

Comments:
- While writing a custom parser was an interesting challenge, but I could have realised sooner that I could parse the input as JSON.
- The `compare()` function could have been written as a standard Python `cmp()` function and returned an integer relative position for `a` versus `b`. This saves on some logic, avoids the mixing of `None` with bools, and avoids the use of `compare_wrapper()` for part 2.
- Structural pattern matching using `match` … `case` could have cleaned up the logic in the `compare()` function further.
- For part 2. a direct sort and search isn't needed. The position of the divider packets in the sorted list is given by the number of packets for which `a < d`, which can be computed directly using the `compare()` function from part 1.
    - see e.g., https://www.reddit.com/r/adventofcode/comments/zkmyh4/2022_day_13_solutions/j00qay8/?context=3

## Revised solution:

In [6]:
import json

def cmp(l, r):
    match l, r:
        case int(), int():  return l - r
        case list(), int(): return cmp(l, [r])
        case int(), list(): return cmp([l], r)
        case list(), list():
            for z in map(cmp, l, r):
                if z: return z
            return len(l) - len(r)

filename = "../data/day13_data.txt"
with open(filename, 'r') as f:
    data = [json.loads(s) for s in f.read().split('\n') if s]
p1 = sum([i for i, (l, r) in enumerate(zip(data[::2], data[1::2]), 1) if cmp(l, r) < 0])
print(p1)
pos1 = sum([1 for d in data if cmp(d, [[2]]) < 0])+1
pos2 = sum([1 for d in data if cmp(d, [[6]]) < 0])+2
p2 = pos1 * pos2
print(p2)

5252
20592
